In [1]:
import requests
import os
from google.auth import default
from google.auth.transport.requests import Request
from google.cloud import optimization_v1
import googlemaps
from dotenv import load_dotenv
import pytz
from datetime import datetime, timezone, timedelta
import pandas
import numpy as np
import io


#assumptions:
# optimal time 7:15 to 9:15
# latest delivery time 9:30


load_dotenv()
proxy_address = "http://127.0.0.1:7890"
# Set the environment variables to specify the proxy
os.environ["HTTP_PROXY"] = proxy_address
os.environ["HTTPS_PROXY"] = proxy_address
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/kaicheng/ProjectsFormal/wheelTrans/secrets/key.json"
os.environ["PROJECT_ID"] = "elderlyhometransportation"
googleMapKey = os.getenv("GOOGLE_MAP_API_KEY")

ROOT_PATH = os.path.dirname(os.path.abspath(''))
ROOT_PATH


'/Users/kaicheng/ProjectsFormal/wheelTrans/src'

In [2]:
def call_sync_api() -> None:
    request_file_name = "resources/request.json"
    fleet_routing_client = optimization_v1.FleetRoutingClient()

    with open(request_file_name) as f:
        fleet_routing_request = optimization_v1.OptimizeToursRequest.from_json(f.read())
        fleet_routing_request.parent = f"projects/elderlyhometransportation"
        fleet_routing_response = fleet_routing_client.optimize_tours(
            fleet_routing_request, timeout=10
        )
        return fleet_routing_response


def get_direction(origin, destination, waypoints):
    waypoints_str = "|".join(["optimize:true"] + waypoints)
    base_url = "https://maps.googleapis.com/maps/api/directions/json"
    google_map_api_key = os.getenv("GOOGLE_MAP_API_KEY")

    params = {
        "destination": destination,
        "origin": origin,
        "waypoints": waypoints_str,
        "key": google_map_api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None


def get_geocoding(gmaps, address):
    result = gmaps.geocode(address)
    if not result or len(result) != 1:
        print(f"Failed to get geocoding for {address}")
        return None
    location = result[0]['geometry']['location']
    lat = location["lat"]
    lng = location["lng"]
    return [lat, lng]


origin = "8408 Garvey Ave. #101 Rosemead, CA 91770"
destination = "8408 Garvey Ave. #101 Rosemead, CA 91770"
address1 = "3843 Maxson Road #226 El Monte, CA 91732"
address2 = "119 Garcelon Ave Apt B Monterey Park, CA 91754"
waypoints = [origin, address1, address2, destination]
# get_direction(origin, destination, waypoints, destination)
# get_geocoding(origin)
# get_geocoding(address1)
# get_geocoding(address2)
# gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
# coordinates = []
# for location in waypoints:
#     coordinates.append(get_geocoding(gmaps, location))
# print(coordinates)
# response = call_sync_api()


In [30]:
weekday = 'M' # 'M', 'T', 'W', 'R', 'F'

def handleInvalidAddress(df):
    print ("toDo")

df = pandas.read_excel("resources/TransportList.xlsx")
validDf = df[df['Address'].notna()]
handleInvalidAddress(df)
attendingAMdf = df[df[weekday].notna() & (df['Trans Method'].isin(['am', 'ampm']))]
cleanDf = attendingAMdf.loc[:, ['MR #','Address','Trans Method', 'M', 'T', 'W', 'R', 'F', 'Notes']]
#dev short
# cleanDf = cleanDf[:10]
cleanDf

toDo


,MR #,Address,Trans Method,M,T,W,R,F,Notes
0,001M,"3843 Maxson Road #226 El Monte, CA 91732",ampm,X,NaN,X,NaN,X,8:00+
1,002F,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+
2,003M,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+
5,007M,"2417 Angelus Ave #306 6266880563 Rosemead, CA ...",ampm,X,NaN,X,NaN,X,NaN
6,009F,"201 1/2 W Newmark Ave Monterey Park, CA 91755",ampm,X,NaN,X,NaN,X,NaN
...,...,...,...,...,...,...,...,...,...
160,167M,"7529 Teresa Ave Rosemead, CA",ampm,X,X,X,X,X,NaN
162,169F,"1206 S 3rd Ave Arcadia,CA",ampm,X,X,X,X,X,NaN
165,172F,"6464 N Vista St San Gabriel, CA ...",ampm,X,X,NaN,X,X,NaN
166,173F,"2511 Hagen Dr, Alhambra, CA,",am,X,X,X,X,X,NaN


In [31]:
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAP_API_KEY"))
columns = cleanDf.columns
geocodedDf = pandas.DataFrame(columns=columns)
invalidGeoCodeDf = pandas.DataFrame(columns=columns)

for ind, row in cleanDf.iterrows():
    latlng = get_geocoding(gmaps,  row['Address'])
    if latlng:
        row['lat'] = latlng[0]
        row['lng'] = latlng[1]
        geocodedDf = pandas.concat([geocodedDf, pandas.DataFrame([row])], ignore_index=True, axis=0)
    else:
        print(f"Failed to get geocoding for {row['Address']}")
        invalidGeoCodeDf = pandas.concat([invalidGeoCodeDf, pandas.DataFrame([row])], ignore_index=True, axis=0)
geocodedDf

Failed to get geocoding for nan
Failed to get geocoding for nan


,MR #,Address,Trans Method,M,T,W,R,F,Notes,lat,lng
0,001M,"3843 Maxson Road #226 El Monte, CA 91732",ampm,X,NaN,X,NaN,X,8:00+,34.072393,-118.010237
1,002F,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+,34.065167,-118.123730
2,003M,"119 Garcelon Ave Apt B Monterey Park, CA 91754",am,X,NaN,NaN,X,X,8:00+,34.065167,-118.123730
3,007M,"2417 Angelus Ave #306 6266880563 Rosemead, CA ...",ampm,X,NaN,X,NaN,X,NaN,34.056449,-118.087469
4,009F,"201 1/2 W Newmark Ave Monterey Park, CA 91755",ampm,X,NaN,X,NaN,X,NaN,34.060019,-118.124452
...,...,...,...,...,...,...,...,...,...,...,...
56,166M,"275 Cordova St Apt 606 Pasadena, CA",ampm,X,X,X,X,X,NaN,34.142452,-118.144862
57,167M,"7529 Teresa Ave Rosemead, CA",ampm,X,X,X,X,X,NaN,34.049553,-118.102412
58,169F,"1206 S 3rd Ave Arcadia,CA",ampm,X,X,X,X,X,NaN,34.126736,-118.023137
59,172F,"6464 N Vista St San Gabriel, CA ...",ampm,X,X,NaN,X,X,NaN,34.116944,-118.083416


In [38]:
from resources.vehicles import vehicles
import json  
import re

def check_time_format(time_str):
    pattern = r'^\d{1,2}:\d{2}[+-]$'
    if re.match(pattern, time_str):
        return True
    else:
        return False
shipments = []
for ind, row in geocodedDf.iterrows():
    shipment = {
        "loadDemands": {
            "weight": {
                "amount": "1"
            }
        },
        "pickups": [
            {
                "arrivalLocation": {
                    "latitude": row['lat'],
                    "longitude": row['lng']
                },
                "duration": "60s",
            }
        ],
        "deliveries": [
            {
                "arrival_location": {
                    "latitude": 34.0623483,
                    "longitude": -118.0859541
                },
                "duration": "10s",
            }
        ]
    }

    note = row['Notes'] if row['Notes'] is not np.nan else ''
    isValidTime = check_time_format(note)
    if isValidTime:
        pickupTime = note.strip('+-')
        if note.endswith('-'):
            startTime = "2024-03-08T7:15:00Z"
            endTime = f"2024-03-08T{pickupTime}:00Z"
        elif note.endswith('+'):
            startTime = f"2024-03-08T{pickupTime}:00Z"
            endTime = "2024-03-08T9:30:00Z"
        shipment['pickups'][0]["timeWindows"] = [{
                "startTime": startTime,
                "endTime": endTime
            }]
        print(f"shipment {ind} start time {startTime} end time {endTime}")

    else:
        if note:  print(f"Invalid time format in the note {note} for {row['MR #']}")


    shipments.append(shipment)

requestDict = {
    "parent": "projects/elderlyhometransportation",
    "model": {
        "shipments": shipments,
        "vehicles": vehicles,
        "global_start_time":"2024-03-08T7:15:00Z",
        "global_end_time":"2024-03-08T9:30:00Z",
        "global_duration_cost_per_hour": "60",
    },
    "populatePolylines": True,
    # "searchMode":2
}
requestJson = json.dumps(requestDict)


shipment 0 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z
shipment 1 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z
shipment 2 start time 2024-03-08T8:00:00Z end time 2024-03-08T9:30:00Z
shipment 11 start time 2024-03-08T7:15:00Z end time 2024-03-08T7:40:00Z
shipment 12 start time 2024-03-08T7:15:00Z end time 2024-03-08T7:40:00Z
shipment 17 start time 2024-03-08T7:15:00Z end time 2024-03-08T8:00:00Z
shipment 18 start time 2024-03-08T7:15:00Z end time 2024-03-08T8:00:00Z


In [41]:
fleet_routing_client = optimization_v1.FleetRoutingClient()
fleetOptimizationRequest = optimization_v1.OptimizeToursRequest.from_json(requestJson)

fleetOptimizationResponse = fleet_routing_client.optimize_tours(
    fleetOptimizationRequest, timeout=100,
)
fleetOptimizationResponse

routes {
  vehicle_start_time {
    seconds: 1709882213
  }
  vehicle_end_time {
    seconds: 1709888803
  }
  visits {
    shipment_index: 18
    is_pickup: true
    start_time {
      seconds: 1709882398
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
    }
    arrival_loads {
      type_: "weight"
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 17
    is_pickup: true
    start_time {
      seconds: 1709882458
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 60
    }
    arrival_loads {
      type_: "weight"
      value: 1
    }
    demands {
      type_: "weight"
      value: 1
    }
  }
  visits {
    shipment_index: 31
    is_pickup: true
    start_time {
      seconds: 1709882811
    }
    load_demands {
      key: "weight"
      value {
        amount: 1
      }
    }
    detour {
      seconds: 204
    }


In [98]:
import polyline

optimizedList=[]
for vehicleRoute in fleetOptimizationResponse.routes:
    points = vehicleRoute.route_polyline.points
    startingMarker = f'34.0623483,-118.0859541'
    for index, visit in enumerate(vehicleRoute.visits):
        if not visit.is_pickup:
            row = pandas.Series()
            row.loc["order"]= index
            row.loc["vehicle"]= vehicleRoute.vehicle_index or 0
            row.loc['MR #'] = "Dropoff"
            row.loc["arrival time"]= visit.start_time
            if optimizedList[-1]['MR #'] != "Dropoff":
                optimizedList.append(row)
            continue

        shipmentId = visit.shipment_index or 0
        row = geocodedDf.loc[shipmentId].copy()
        row.loc["arrival time"]= visit.start_time
        row.loc["vehicle"]= vehicleRoute.vehicle_index or 0
        row.loc["order"]= index
        row.loc["routePolyline"]= points
        optimizedList.append(row)
    
optimizedDf = pandas.DataFrame(optimizedList)
optimizedDf

,MR #,Address,Trans Method,M,T,W,R,F,Notes,lat,lng,arrival time,vehicle,order,routePolyline
18,066M,"3058 Brighton Ave Rosemead, CA 91770",ampm,X,X,X,X,X,8:00-,34.064323,-118.097957,2024-03-08 07:19:58+00:00,0,0,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
17,065F,"3058 Brighton Ave Rosemead, CA 91770",ampm,X,X,X,X,X,8:00-,34.064323,-118.097957,2024-03-08 07:20:58+00:00,0,1,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
31,105M,"234 N Rural Dr A106 Monterey Park, CA",ampm,X,X,X,X,X,NaN,34.064866,-118.113868,2024-03-08 07:26:51+00:00,0,2,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
39,127F,"234n Rural Dr Apt A320, Monterey Park, CA 9175...",ampm,X,X,X,X,X,NaN,34.064866,-118.113868,2024-03-08 07:27:51+00:00,0,3,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
28,097M,"526 N Nicholson Ave Apt F Monterey Park, CA",ampm,X,NaN,X,NaN,X,NaN,34.068233,-118.118463,2024-03-08 07:31:52+00:00,0,4,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,138M,"10124 Valley Blvd Unit 132 El Monte, CA",ampm,X,NaN,X,NaN,X,NaN,34.077716,-118.050238,2024-03-08 08:25:38+00:00,3,5,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
Unnamed 9,Dropoff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-08 08:35:40+00:00,3,6,NaN
15,057F,"8355 Sheffield Rd San Gabriel, CA 91775",ampm,X,NaN,X,NaN,NaN,NaN,34.117379,-118.087234,2024-03-08 08:49:07+00:00,3,7,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...
59,172F,"6464 N Vista St San Gabriel, CA ...",ampm,X,X,NaN,X,X,NaN,34.116944,-118.083416,2024-03-08 08:51:10+00:00,3,8,ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA...


In [138]:
import polyline
from PIL import Image, ImageDraw, ImageFont, ImageOps
from resources.vehicles import vehicles


def drawToImage(byteImage, filename, waypoints):
    image = Image.open(io.BytesIO(byteImage))
    image = image.convert("RGB")
    padded_image = ImageOps.expand(image, (800, 0,0,500), fill="white")
    draw = ImageDraw.Draw(padded_image)
    text ='\n'.join(waypoints)
    font = ImageFont.load_default()
    font_size = 15
    font = font.font_variant(size=font_size)
    text_position = (10, 10)
    text_color = (0, 0, 0)
    draw.text(text_position, text, fill=text_color, font=font)

    padded_image.save(f"output/{filename}.png")

def generate_static_map(origin, destination, routePolyline, waypoints,filename):
    startingMarker = f'34.0623483,-118.0859541'
    # Add markers for each waypoint with labels
    markers = []
    label =1
    for i, waypoint in enumerate(waypoints):
        if (np.isnan(waypoint[0])): 
            label = label + 1
            continue
        markers.append(f"markers=label:{label}|color:red|{waypoint[0]},{waypoint[1]}")
    markers_str = '&'.join(markers)
    static_map_url = f'https://maps.googleapis.com/maps/api/staticmap?size=1200x800&path=enc:{routePolyline}&{markers_str}&markers=color:blue|{startingMarker}&key={googleMapKey}'
    print (static_map_url)
    # Download the map image
    map_image = requests.get(static_map_url)
    return map_image.content


origin = (34.0623483, -118.0859541)  # Los Angeles, CA
destination = (34.0623483, -118.0859541)  # Los Angeles, CA
vehiclesIds = optimizedDf['vehicle'].unique()
for vehicleId in vehiclesIds:
    number_of_seats = vehicles[vehicleId]['loadLimits']['weight']['maxLoad']
    sortedDf = optimizedDf[optimizedDf['vehicle'] == vehicleId].sort_values(by='order')
    waypoints = [[lat, lon] for lat, lon in sortedDf[['lat', 'lng']].values]
    routePolyline = sortedDf['routePolyline'].values[0]
    byteImage=generate_static_map(origin=origin, destination=destination, routePolyline=routePolyline,waypoints=waypoints, filename=f'map_with_route{vehicleId}.png' )
    #write to txt
    displayWaypointInfoList= sortedDf[['order', 'MR #', 'Address', 'arrival time', 'Notes']].to_csv(f'output/waypointInfo{vehicleId}.csv', header=True, index=False)
    # save to image
    with open(f"output/{vehicleId}.png", "wb") as f:
        f.write(byteImage)
    #draw overlay
    header=[f'vehicle Id:{vehicleId +1}       number of seats: {number_of_seats}\n\n\n\n\n'
            ,'order        '+'MR #        ' +' Address                                                          '+    'timeOfArrival        '+    'notes\n'] 
    displayWaypointInfo=header+[f"{order}        {MR}        {Address if Address is not np.nan else 'N/A'}            {arrivalTime.strftime('%H:%M')}            {Notes if Notes is not np.nan else 'N/A'}" for order, MR, Address, arrivalTime, Notes in sortedDf[['order', 'MR #', 'Address', 'arrival time', 'Notes']].values]
    drawToImage(byteImage, f'vehicle_{vehicleId}_seat_{number_of_seats}', displayWaypointInfo)

[34.0643228, -118.097957] 1
[34.0643228, -118.097957] 1
[34.0648659, -118.113868] 1
[34.0648659, -118.113868] 1
[34.0682325, -118.1184628] 1
[34.0673336, -118.1205658] 1
[34.0663829, -118.1221047] 1
[34.072393, -118.0102374] 2
[34.072393, -118.0102374] 2
[34.072393, -118.0102374] 2
[34.0793334, -118.0613962] 2
[34.0802862, -118.0637524] 2
[34.082718, -118.0706302] 2
[34.0762409, -118.0546959] 2
[34.06028370000001, -118.0540782] 3
[34.0587468, -118.0467618] 3
[34.0652888, -118.0477687] 3
[34.0679616, -118.0572812] 3
[34.0640168, -118.0607569] 3
[34.0564491, -118.0874686] 3
[34.0564491, -118.0874686] 3
https://maps.googleapis.com/maps/api/staticmap?size=1200x800&path=enc:ux{nEjvvoU}@?@hAAzAIPAPAZ?Z@h@JX@x@?X@rB?jAGDA@?B?T?V?R?p@?P?~A?rA?T@xAAfB?p@?~@@jB?hA?J?vB?PAd@@D@HFX?t@?z@AtB?nB@hBGh@?rC?@?`BgA?C?C?E?eE@[@ZAdEAD?B?B?fA??~@?^?tA?B?d@?b@HH?fC?R?tA?dA?tA?H?tA?\@P?fB?p@?lB?tB@dBAr@@bD?n@@fB@~@@vA?L@hA@PIZ?dC?vB@tF?bD?tA?`A@v@A^?b@?d@?~@@j@?nBuB?_@?M?S?G?G?s@?w@?[?s@@C?i@?S?S?]?Q?g@?O?Q?

In [125]:
markers = []
for i, waypoint in enumerate(waypoints):
    if (waypoints[0]==np.nan): continue
    markers.append(f"markers=label:{i+1}|color:red|{waypoint[0]},{waypoint[1]}")
'&'.join(markers)


'markers=label:1|color:red|34.0643228,-118.097957&markers=label:2|color:red|34.0643228,-118.097957&markers=label:3|color:red|34.0648659,-118.113868&markers=label:4|color:red|34.0648659,-118.113868&markers=label:5|color:red|34.0682325,-118.1184628&markers=label:6|color:red|34.0673336,-118.1205658&markers=label:7|color:red|34.0663829,-118.1221047&markers=label:8|color:red|nan,nan&markers=label:9|color:red|34.072393,-118.0102374&markers=label:10|color:red|34.072393,-118.0102374&markers=label:11|color:red|34.072393,-118.0102374&markers=label:12|color:red|34.0793334,-118.0613962&markers=label:13|color:red|34.0802862,-118.0637524&markers=label:14|color:red|34.082718,-118.0706302&markers=label:15|color:red|34.0762409,-118.0546959&markers=label:16|color:red|nan,nan&markers=label:17|color:red|34.06028370000001,-118.0540782&markers=label:18|color:red|34.0587468,-118.0467618&markers=label:19|color:red|34.0652888,-118.0477687&markers=label:20|color:red|34.0679616,-118.0572812&markers=label:21|colo